In [ ]:
# Imports
import numpy as np

In [ ]:
def ghmm_learn(Yseq,N,Ainit=None):
    M = np.shape(Yseq[0])[1]
    P0 = np.ones([N,1])/N;
    
    if Ainit is None:
        A = np.random.rand(N,N) + 2 * np.eye(N)
        for l in range(N):
            A[l,:] = A[l,:]/np.sum(A[l,:]);
    else:
        A = Ainit
        
    Nseq = len(Yseq)
    
    datatmp = Yseq[0]
    for seq in range(1,Nseq):
        datatmp = np.append(datatmp,Yseq[seq],axis=0)
        
    M = np.shape(datatmp)[1]
    T = np.shape(datatmp)[0]
    T1 = int(np.floor(T/N))
    
    mu = np.zeros([N,M])
    sigma = np.zeros([N,M,M])

    for i in range(N):
        mu[i,:] = np.mean(datatmp[i*T1:(i+1)*T1-1,:],axis=0)
        sigma[i,:,:] = np.cov(np.transpose(datatmp))
        
    for itr in range(25):
        params = ghmm_em_obs(Yseq,N,A,P0,mu,sigma)
        mu = params[0]
        A = ghmm_em_trans(Yseq,N,A,P0,mu,sigma)
    return(P0,A,mu,sigma)

In [ ]:
def ghmm_fwd(Y,A,P0,mu,sigma):
    N = np.shape(A)[0]
    i = np.ones([1,N])
    T = np.shape(Y)[0]
    M = np.shape(Y)[1]
    alpha = np.zeros([np.shape(P0)[0],T])
    
    Pys = np.zeros([N,T])
    # Forward 
    for j in range(N):
        sigmai = np.reshape(sigma[j,:,:],(M,M),order='F')
        for t in range(T):
            Pys[j,t] = np.sqrt(1/(((2*np.pi)**M)*np.linalg.det(sigmai)))*np.exp(np.dot(-0.5*(Y[t,:]-mu[j,:]),np.dot(np.linalg.inv(sigmai),(Y[t,:]-mu[j,:]).reshape(-1,1))))

    alpha0 = np.multiply(P0,Pys[:,0].reshape(-1,1))
    scale = np.zeros([T,1])
    scale[0] = np.sum(alpha0)    
    alpha[:,0] = (alpha0 / scale[0]).flatten()
    
    for t in range(1,T):
        ######################################################################
        # You need to insert a line of code into ghmm_fwd.m that computes    #
        # alpha0 from transition probs A, Gaussian probs Pys, and alpha[t-1] #
        ######################################################################
        alpha0 = ???
        scale[t] = np.sum(alpha0)
        alpha[:,t] = alpha0 / scale[t]
        
    scale = np.log(scale)
    return (alpha,scale)

In [ ]:
def ghmm_bwd(Y,A,P0,mu,sigma,scale):
    scale = np.exp(scale)
    N = np.shape(A)[0];
    T = np.shape(Y)[0]
    M = np.shape(Y)[1]
    
    Pys = np.zeros([N,T])
    # Forward
    for j in range(N):
        sigmai = np.reshape(sigma[j,:,:],[M,M],order='F')
        for t in range(T):
            Pys[j,t] = np.sqrt(1/(((2*np.pi)**M)*np.linalg.det(sigmai)))*np.exp(np.dot(-0.5*(Y[t,:]-mu[j,:]),np.dot(np.linalg.inv(sigmai),(Y[t,:]-mu[j,:]).reshape(-1,1))))
            
    # Backward
    beta = np.zeros([N,T])
    beta[:,T-1] = (np.ones([N,1])/scale[T-1]).flatten()
    for t in range(T-2,-1,-1):
        
        ###################################################################
        # You need to insert a line of code into ghmm_bwd.m that computes #
        # beta from transition probs A, Gaussian probs Pys, and beta(t+1) #
        ###################################################################
        beta[:,t] = ???
    beta = np.log(beta+1.0e-10)
    
    return beta

In [ ]:
def ghmm_prob(alpha,beta,scale,A,mu,sigma,Y):
    scale = np.exp(scale)
    
    N = np.shape(A)[0]
    i = np.ones([1,N])
    T = np.shape(Y)[0]
    M = np.shape(Y)[1]
    
    Pys = np.zeros([N,T])

    for j in range(N):
        sigmai = np.reshape(sigma[j,:,:],(M,M),order='F')
        for t in range(T):
            Pys[j,t] = np.sqrt(1/(((2*np.pi)**M)*np.linalg.det(sigmai)))*np.exp(np.dot(-0.5*(Y[t,:]-mu[j,:]),np.dot(np.linalg.inv(sigmai),(Y[t,:]-mu[j,:]).reshape(-1,1))))
    Pxy = np.zeros([T,N])
    for t in range(T):
        Pxy0 = np.multiply(alpha[:,t],np.exp(beta[:,t])*scale[t])

        #####################################################################
        # You need to insert a line of code into ghmm_prob.m that computes')#
        # p(x|y) from p(x,y), where x is state index, y is observation')    #
        #####################################################################
        Pxy[t,:] = ???
        
    Pxxy = np.zeros([T,N,N])
    for t in range(T-1):
        Pxxy0 = np.multiply(alpha[:,t].reshape(-1,1) * np.transpose(np.multiply(np.exp(beta[:,t+1]),Pys[:,t+1])),A)
        ####################################################################
        # You need to insert a line of code into ghmm_prob.m that computes #
        # p(x(t),x(t+1)|Y) from p(x(t),x(t+1),Y)                           #
        ####################################################################
        Pxxy[t,:,:]= ???
    return(Pxy,Pxxy)

In [ ]:
def ghmm_em_trans(Yseq,N,A,P0,mu,sigma):
    N_seq = np.max(np.shape(Yseq))
    eps = 0
    MAX_ITR = 20
    Naseq = np.zeros([N,N])
    
    for seq in range(N_seq):
        Y = Yseq[seq]
        M = np.shape(Y)[1]
        T = np.shape(Y)[0]
        fwd = ghmm_fwd(Y,A,P0,mu,sigma)
        alpha = fwd[0]
        scale = fwd[1]
        beta = ghmm_bwd(Y,A,P0,mu,sigma,scale)
        probs = ghmm_prob(alpha,beta,scale,A,mu,sigma,Y)
        Pxy = probs[0]
        Pxxy = probs[1]
        
        Na = np.zeros([N,N])
        for i in range(N):
            for j in range(N):
                Na[i,j] = np.sum(Pxxy[:,i,j])
        
        Na = Na+eps
        Naseq = Naseq + Na
    Anew = np.zeros([N,N])
    for l in range(N):
        ######################################################################
        # You need to insert a line of code into ghmm_em_trans that computes #
        # the new transition probabilities A[l,m] from the corresponding     #
        # soft-count matrix Na[l,m]                                          #
        ######################################################################
        Anew[l,:] = ???
    return Anew

In [ ]:
def ghmm_em_obs(Yseq,N,A,P0,mu,sigma):
    N_seq = np.max(np.shape(Yseq))
    eps = 1e-5
    M = np.shape(Yseq[0])[1]
    museq_unnorm = np.zeros([N,M])
    museq_norm = np.zeros([N,M])
    sigmaseq_unnorm = np.zeros([N,M,M])
    sigmaseq_norm = np.zeros([N,1])
    Pxyseq = list([] for _ in range(N_seq))
    Pxxyseq = list([] for _ in range(N_seq))
    llseq = list([] for _ in range(N_seq))
    
    for seq in range(N_seq):
        Y = Yseq[seq]
        M = np.shape(Y)[1]
        T = np.shape(Y)[0]

        fwd = ghmm_fwd(Y,A,P0,mu,sigma)
        alpha = fwd[0]
        scale = fwd[1]
        beta = ghmm_bwd(Y,A,P0,mu,sigma,scale)
        probs = ghmm_prob(alpha,beta,scale,A,mu,sigma,Y)
        Pxy = probs[0]
        Pxxy = probs[1]
        Pxyseq[seq] = Pxy
        Pxxyseq[seq] = Pxxy
        llseq[seq] = np.exp(np.sum(scale))
        
        for i in range(N):
            Pxy0 = Pxy[:,i].reshape(-1,1)
            Pxy0 = np.sum(np.multiply(np.repeat(Pxy0,M,axis=1),Y),axis=0)+eps
            museq_unnorm[i,:] = np.add(museq_unnorm[i,:],Pxy0)
            museq_norm[i,:] = np.add(museq_norm[i,:],np.sum(Pxy[:,i])+eps)
            
    mu = np.divide(museq_unnorm,museq_norm)
    sigmaseq_unnorm = np.zeros([N,M,M])
    sigmaseq_norm = np.zeros([N,1])
    for seq in range(N_seq):
        Y = Yseq[seq]
        M = np.shape(Y)[1]
        T = np.shape(Y)[0]

        fwd = ghmm_fwd(Y,A,P0,mu,sigma)
        alpha = fwd[0]
        scale = fwd[1]
        beta = ghmm_bwd(Y,A,P0,mu,sigma,scale)
        probs = ghmm_prob(alpha,beta,scale,A,mu,sigma,Y)
        Pxy = probs[0]
        Pxxy = probs[1]
        Pxyseq[seq] = Pxy
        Pxxyseq[seq] = Pxxy
        llseq[seq] = np.exp(np.sum(scale));
    
        for i in range(N):
            Pxy0 = Pxy[:,i]
            sigmatmp = np.zeros([M,M])
            for t in range(T):
                sigmatmp = np.add(sigmatmp, np.multiply(np.transpose(Y[t,:] - mu[i,:]).reshape(-1,1), np.multiply(Y[t,:] - mu[i,:],Pxy0[t])))
                sigmaseq_norm[i] = sigmaseq_norm[i] + Pxy0[t]
            
            sigmaseq_unnorm[i,:,:] = sigmaseq_unnorm[i,:,:] + np.reshape(sigmatmp,[1,M,M],order='F')
    for i in range(N):
        sigma[i,:,:] = np.divide(sigmaseq_unnorm[i,:,:],sigmaseq_norm[i])
    return(mu,sigma)                                              